 ### Install Required Libraries

In [ ]:
!pip install datasets
!pip install kagglehub
!pip install transformers
!pip install evaluate
!pip install transformers datasets evaluate huggingface_hub -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "L

## Importing Required Librires

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import kagglehub
import evaluate
import torch as t
import transformers
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoModelForSequenceClassification,AutoTokenizer,DataCollatorWithPadding,Trainer,TrainingArguments, EarlyStoppingCallback
from datasets import load_dataset,DatasetDict,ClassLabel,Features,Value

In [ ]:
t.cuda.is_available()

True

## Installing pre_trined Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-multilingual-cased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
tokenizer

BertTokenizerFast(name_or_path='google-bert/bert-base-multilingual-cased', vocab_size=119547, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

### lodaing data into dict

In [ ]:
data=load_dataset('Sanath369/Telugu_sentiment_sentences')

README.md:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

training_data.csv:   0%|          | 0.00/5.70M [00:00<?, ?B/s]

test_data.csv:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/24599 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7033 [00:00<?, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 24599
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7033
    })
})

In [ ]:
#split the train data into train and Validation
main_data1=data['train'].train_test_split(test_size=0.2)

In [ ]:
main_data1

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 19679
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 4920
    })
})

In [ ]:
# creating the final dataset
final_data={'train':main_data1['train'],'test':data['test'],'validation':main_data1['test']}

In [ ]:
final_data

{'train': Dataset({
     features: ['text', 'label'],
     num_rows: 19679
 }),
 'test': Dataset({
     features: ['text', 'label'],
     num_rows: 7033
 }),
 'validation': Dataset({
     features: ['text', 'label'],
     num_rows: 4920
 })}

### Prepare data for Model Input

In [ ]:
#Text Preprocessing: Handling Empty Reviews & Tokenization
def pre_processing(batch):
    review_text = []
    for i in batch['text']:
        if i and i.strip():
            review_text.append(i)
        else:
            review_text.append("[EMPTY]")
    tokenizers = tokenizer(
        review_text,
        truncation=True,
        padding='max_length',
        max_length=512
    )
    return tokenizers

In [ ]:
final_data = DatasetDict(final_data)

In [ ]:
#passing the preproccing funtion
final_tokenized_data=final_data.map(pre_processing,batched=True,remove_columns=['text'])

Map:   0%|          | 0/19679 [00:00<?, ? examples/s]

Map:   0%|          | 0/7033 [00:00<?, ? examples/s]

Map:   0%|          | 0/4920 [00:00<?, ? examples/s]

In [ ]:
final_tokenized_data

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 19679
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7033
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4920
    })
})

 ### Load Pretrained BERT Model for Sequence Classification

In [ ]:
model=AutoModelForSequenceClassification.from_pretrained('google-bert/bert-base-multilingual-cased', num_labels=3)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Define Custom Metrics for Trainer Evaluation

In [ ]:
def compute_metrics(eval_pred):
    # Unpack logits and labels
    logits, labels = eval_pred

    # Convert logits to NumPy if not already
    if not isinstance(logits, np.ndarray):
        logits = np.array(logits)

    # Convert labels to NumPy if not already
    if not isinstance(labels, np.ndarray):
        labels = np.array(labels)

    # Compute predictions by taking argmax along the last axis
    predictions = np.argmax(logits, axis=-1)

    # Compute accuracy
    acc = accuracy_score(labels, predictions)

    # Compute F1-score for 3-class sentiment analysis (weighted for imbalanced classes)
    f1 = f1_score(labels, predictions, average="weighted")  # Use "macro" for equal class weighting

    return {"accuracy": acc, "f1": f1}
    predictions = logits.argmax(axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")  # or "macro" / "binary"
    return {"accuracy": acc, "f1": f1}


### Define TrainingArguments for Model Fine-Tuning

In [ ]:
ta=TrainingArguments(eval_strategy ="epoch",
                  learning_rate=2e-5,save_strategy="epoch",
                  logging_dir="/content/.config",
                  num_train_epochs=5,output_dir="/content/sentiment-bert-model",
                  per_device_train_batch_size=16,per_device_eval_batch_size=16,fp16=True,
                  metric_for_best_model="eval_loss")

### Trainer Setup for Fine-Tuning

In [ ]:
tr=Trainer(model=model,
        args=ta,
        train_dataset=final_tokenized_data["train"],
        eval_dataset=final_tokenized_data["validation"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[
        EarlyStoppingCallback(
            early_stopping_patience=2,       # stop after 3 epochs of no real improvement
            early_stopping_threshold=0.01)])

<ipython-input-20-e1c629b8eee0>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  tr=Trainer(model=model,


### Start Model Training

In [ ]:
tr.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pavanmuthyala45 (pavanmuthyala45-innomatics-research-labs) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.834400,0.765251,0.670935,0.654113
2,0.705000,0.737157,0.678659,0.671936
3,0.593800,0.759878,0.681504,0.674119
4,0.481500,0.819187,0.685772,0.684734


TrainOutput(global_step=4920, training_loss=0.6612224547843623, metrics={'train_runtime': 2453.4868, 'train_samples_per_second': 40.104, 'train_steps_per_second': 2.507, 'total_flos': 2.071123578987725e+16, 'train_loss': 0.6612224547843623, 'epoch': 4.0})

In [ ]:
results = tr.evaluate(eval_dataset=final_tokenized_data['test'])
print("Test Evaluation Results:", results)

Test Evaluation Results: {'eval_loss': 0.8279711008071899, 'eval_accuracy': 0.6729702829517986, 'eval_f1': 0.6724756557429611, 'eval_runtime': 56.0613, 'eval_samples_per_second': 125.452, 'eval_steps_per_second': 7.849, 'epoch': 4.0}


### Saving the model


In [ ]:
tr.model.save_pretrained("/content/telugu_sentiment_bert_model_new")
tokenizer.save_pretrained("/content/telugu_sentiment_bert_model_new")

('/content/telugu_sentiment_bert_model_new/tokenizer_config.json',
 '/content/telugu_sentiment_bert_model_new/special_tokens_map.json',
 '/content/telugu_sentiment_bert_model_new/vocab.txt',
 '/content/telugu_sentiment_bert_model_new/added_tokens.json',
 '/content/telugu_sentiment_bert_model_new/tokenizer.json')

### Loading Finetuned  Model

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("/content/telugu_sentiment_bert_model_new")
tokenizer = AutoTokenizer.from_pretrained("/content/telugu_sentiment_bert_model_new")

# Steps to Upload Model to Hugging Face

### 1. Create a Model Repository on Hugging Face
   - Visit [Hugging Face](https://huggingface.co), log in, and create a new model repository via your profile.
   - Set the repository name and visibility (Public or Private) as desired.

### 2. Obtain an API Key from Hugging Face
   - Go to your Hugging Face account settings.
   - Select the option to create a new API key with **write** permissions.
   - Copy and securely store the generated API token.

### 3. Install the Hugging Face Hub Library
   - Install the `huggingface_hub` package to interact with Hugging Face.
   ```bash
   pip install huggingface_hub

### 4. Log in to Hugging Face in Your Environment
  - Use the notebook_login function to authenticate in a Jupyter or Colab notebook.

        from huggingface_hub import notebook_login
        notebook_login()
   - A pop-up will appear in Colab/Jupyter to enter your Hugging Face API token
   - Enter the API token obtained in Step 2.

### 5. Save and Name Your Model
   - Save your model and tokenizer locally, ensuring the name matches the repository name created on Hugging Face.


          from transformers import AutoTokenizer, AutoModelForSequenceClassification

          model.save_pretrained("your_model_directory")
          tokenizer.save_pretrained("your_model_directory")

### 6. Push Your Model to Hugging Face
   - Use the Hugging Face API to upload your model and tokenizer to the repository.

          from huggingface_hub import HfApi, HfFolder, Repository
          from transformers import AutoTokenizer, AutoModelForSequenceClassification

          # Push model and tokenizer to the repository
          model.push_to_hub("username/your-model-name")
          tokenizer.push_to_hub("username/your-model-name")

### 7. Create a Pipeline to Test the Model
   - Use the pipeline helper to load and test your uploaded model.
python

          from transformers import pipeline

          pipe = pipeline("text-classification", model="username/your-model-name")

### 8. Verify the Model Upload

  - Visit your model repository on Hugging Face (e.g., https://huggingface.co/username/your-model-name) to confirm the model and tokenizer were uploaded successfully.

  - Test the pipeline to ensure the model works as expected.


### Uploading model to hugging face Model

In [ ]:
from huggingface_hub import notebook_login

notebook_login()  # A pop-up will appear in Colab asking for your HF tokena



In [ ]:
from huggingface_hub import HfApi, HfFolder, Repository
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Push model and tokenizer
model.push_to_hub("Mpavan45/Telugu_Sentimental_Analysis")
tokenizer.push_to_hub("Mpavan45/Telugu_Sentimental_Analysis")a


model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Mpavan45/Telugu_Sentimental_Analysis/commit/b7c5fe7fa6512c93b492683f6694c0f3bfbc8322', commit_message='Upload tokenizer', commit_description='', oid='b7c5fe7fa6512c93b492683f6694c0f3bfbc8322', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Mpavan45/Telugu_Sentimental_Analysis', endpoint='https://huggingface.co', repo_type='model', repo_id='Mpavan45/Telugu_Sentimental_Analysis'), pr_revision=None, pr_num=None)

### Importing finetuned model and creating pipe-line

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="Mpavan45/Telugu_Sentimental_Analysis")

config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

Device set to use cuda:0


### Testing the model with different Telugu texts

In [ ]:
# Example usage with different Telugu texts:
test_texts = [
     "ఈ సినిమా చాలా బాగుంది",  # This movie is very good
    "నేను ఈ పుస్తకాన్ని చాలా ఇష్టపడ్డాను",  # I liked this book very much
     "ఈ ఆహారం చాలా చెడుగా ఉంది",  # This food is very bad
    "నాకు ఈ రోజు చాలా సంతోషంగా ఉంది",  # I am very happy today
     "నేను ఈ వార్తలకు చాలా బాధపడ్డాను", # I felt very sad for this news
     "ఈ వాతావరణం చాలా అద్భుతంగా ఉంది", # This weather is very wonderful
     "నేను ఈ పనిని పూర్తి చేయలేకపోతున్నాను", # I can not complete this work
     "ఈ కారు చాలా వేగంగా ఉంది",  # This car is very fast
     "ఈ పాట చాలా అందంగా ఉంది",  # This song is very beautiful
     "ఈ ప్రదేశం చాలా ప్రశాంతంగా ఉంది", # This place is very calm
    "నేను చాలా అలసిపోయాను", # I am very tired
     "ఈ సమస్యకు పరిష్కారం లేదు", # There is no solution to this problem
     "నేను ఈ సమాచారాన్ని నమ్మలేకపోతున్నాను", # I can not believe this information
     "ఈ వ్యక్తి చాలా మంచివాడు", # This person is very good
     "ఈ పనిని త్వరగా పూర్తి చేయాలి" #This work should be completed quickly
    # # Add more examples here...
]

for text in test_texts:
    result = pipe(text)
    print(f"Text: {text}")
    print(f"Result: {result}")
    print("-" * 20)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Text: ఈ సినిమా చాలా బాగుంది
Result: [{'label': 'LABEL_2', 'score': 0.9709675312042236}]
--------------------
Text: నేను ఈ పుస్తకాన్ని చాలా ఇష్టపడ్డాను
Result: [{'label': 'LABEL_2', 'score': 0.9643860459327698}]
--------------------
Text: ఈ ఆహారం చాలా చెడుగా ఉంది
Result: [{'label': 'LABEL_0', 'score': 0.4701785445213318}]
--------------------
Text: నాకు ఈ రోజు చాలా సంతోషంగా ఉంది
Result: [{'label': 'LABEL_2', 'score': 0.9663376212120056}]
--------------------
Text: నేను ఈ వార్తలకు చాలా బాధపడ్డాను
Result: [{'label': 'LABEL_0', 'score': 0.9451944231987}]
--------------------
Text: ఈ వాతావరణం చాలా అద్భుతంగా ఉంది
Result: [{'label': 'LABEL_2', 'score': 0.9697619676589966}]
--------------------
Text: నేను ఈ పనిని పూర్తి చేయలేకపోతున్నాను
Result: [{'label': 'LABEL_1', 'score': 0.6042336821556091}]
--------------------
Text: ఈ కారు చాలా వేగంగా ఉంది
Result: [{'label': 'LABEL_2', 'score': 0.8727412223815918}]
--------------------
Text: ఈ పాట చాలా అందంగా ఉంది
Result: [{'label': 'LABEL_2', 'score': 0